In [1]:
import fastrepl.repl as fastrepl

fastrepl.LLMCache.disable()
fastrepl.DEBUG(0)

In [2]:
from IPython.display import clear_output

In [3]:
from datasets import load_dataset

# We have dataset for meta-eval
dataset = load_dataset("yelp_review_full", split="test")
dataset = dataset.shuffle(seed=8)
dataset = dataset.select(range(50))
dataset = dataset.rename_column("text", "input")
dataset = dataset.map(
    lambda row: {"reference": row["label"] + 1, "input": row["input"]},
    remove_columns=["label"],
)

In [4]:
def print_metric(metric_name, predictions, references):
    metric = fastrepl.load_metric(metric_name)
    result = metric.compute(predictions=predictions, references=references)
    print(f"{metric_name}: {result[metric_name]}")

In [5]:
clear_output(wait=True)

eval1 = fastrepl.Evaluator(
    pipeline=[
        fastrepl.LLMGradingHead(
            model="gpt-3.5-turbo",
            context="You will get a input text from Yelp review. Grade user's satisfaction from 1 to 5.",
            number_from=1,
            number_to=5,
            position_debias_strategy="shuffle",
        )
    ]
)


def to_int(example):
    example["prediction"] = int(example["prediction"])
    return example


result1 = fastrepl.LocalRunner(evaluator=eval1, dataset=dataset).run()
result1 = result1.map(to_int)

print_metric("accuracy", result1["prediction"], result1["reference"])
print_metric("mse", result1["prediction"], result1["reference"])
print_metric("mae", result1["prediction"], result1["reference"])

result1.to_pandas()

Output()

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/llm.py:140: UserWarning: gpt-3.5-turbo completion truncated due to 
length
  warnings.warn(f"{maybe_fallback} completion truncated due to length")

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

accuracy: 0.28
mse: 2.12
mae: 1.12


,input,reference,prediction
0,Stayed at the Wm Penn down the street for a we...,5,4
1,We tried out the lunch specials and found them...,4,4
2,Should have known better than to eat in a plac...,1,1
3,This place has sure changed...and not for the ...,2,1
4,I've been bringing my son to the owner Michael...,5,5
5,The trip to the location takes two busses and ...,1,1
6,This restaurant was suggested to me by a frien...,4,4
7,This place is unique because you are sitting o...,3,1
8,"Whew, $10 for a whiskey ginger ale and $18 for...",3,4
9,"Loove me some Pei Wei! Boyfriend isn't a fan, ...",4,1


In [6]:
clear_output(wait=True)

# Same as above, just provide simple references
eval2 = fastrepl.Evaluator(
    pipeline=[
        fastrepl.LLMGradingHead(
            model="gpt-3.5-turbo",
            context="You will get a input text from Yelp review. Grade user's satisfaction from 1 to 5.",
            number_from=1,
            number_to=5,
            position_debias_strategy="shuffle",
            references=[("this is the best!", "5"), ("this is the worst!", "1")],
        )
    ]
)


def to_int(example):
    example["prediction"] = int(example["prediction"])
    return example


result2 = fastrepl.LocalRunner(evaluator=eval2, dataset=dataset).run()
result2 = result2.map(to_int)

print_metric("accuracy", result2["prediction"], result2["reference"])
print_metric("mse", result2["prediction"], result2["reference"])
print_metric("mae", result2["prediction"], result2["reference"])

result2.to_pandas()

Output()

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/llm.py:140: UserWarning: gpt-3.5-turbo completion truncated due to 
length
  warnings.warn(f"{maybe_fallback} completion truncated due to length")

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

accuracy: 0.66
mse: 0.34
mae: 0.34


,input,reference,prediction
0,Stayed at the Wm Penn down the street for a we...,5,4
1,We tried out the lunch specials and found them...,4,4
2,Should have known better than to eat in a plac...,1,2
3,This place has sure changed...and not for the ...,2,2
4,I've been bringing my son to the owner Michael...,5,5
5,The trip to the location takes two busses and ...,1,1
6,This restaurant was suggested to me by a frien...,4,4
7,This place is unique because you are sitting o...,3,3
8,"Whew, $10 for a whiskey ginger ale and $18 for...",3,4
9,"Loove me some Pei Wei! Boyfriend isn't a fan, ...",4,4
